## Data wrangling to reproduce Istat-based KPIs

In [ ]:
from enum import Enum
from os import path, listdir

import numpy as np
import pandas as pd
import geopandas as gpd
import geopy, geopy.distance
import shapely
from sklearn import gaussian_process

from matplotlib import pyplot as plt 
import seaborn as sns
plt.rcParams['figure.figsize']= (20,14)

In [ ]:
## TODO: find way to put this into some global settings
import os
import sys
nb_dir = os.path.dirname(os.getcwd())
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from references import common_cfg

In [ ]:
selectedCity = common_cfg.cityList[0]
print(selectedCity)

In [ ]:
# load sezione di censimento data for the selected city
istatData = common_cfg.get_istat_cpa_data(selectedCity)

In [ ]:
istatData

In [ ]:
quartiereData = istatData.groupby(common_cfg.IdQuartiereColName).sum()
quartiereData

In [ ]:
metaData = common_cfg.get_istat_metadata()
metaData

## Indici

1. Percentuale residenti stranieri
    - Sul totale dei residenti
2. Percentuali di provenienza deglio stranieri:
    - Europa
    - Africa
    - Asia
    - America
    - Oceania
    
3. Indice di vecchiaia
    - Sul totale popolazione
    - Tra gli uomini
    - Tra le donne
    
4. Indice di popolazione attiva
    - Sul totale popolazione
    - Tra gli uomini
    - Tra le donne